In [1]:
import json
import pandas as pd
import os
import sys
import pickle
import numpy as np
from random import random
import math
import torch
import torchvision 
import torch.nn.functional as F  
import torchvision.datasets as datasets  
import torchvision.transforms as transforms  
from torch import optim  
from torch import nn  
from torch.utils.data import DataLoader  
from tqdm import tqdm  
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
parentPath = '/content/drive/MyDrive/PhD/Fragle_TSS/Tested_Algorithms/Large Bin Based Modeling/Dataset/Sig_10_Mb_10_Splits'
os.chdir(parentPath)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, criterion, optimizer = None, None, None
train_indices, test_indices = [], []
train_samples, test_samples, max_arr = None, None, None
train_meta_info, test_meta_info = None, None

json_file = open('../../meta_info_files/split_patient_wise.json')
dic = json.load(json_file)
myPath = os.getcwd()

thresholds = [0.05, 0.03, 0.02, 0.01, 0.005, 0.001]
MAE_dic, SN_dic, SP_dic = {}, {}, {}
for thr in thresholds:
    MAE_dic[thr] = []
    SN_dic[thr] = []
    SP_dic[thr] = []

In [4]:
loaded_dict = {}
with open('train_samples.pkl', 'rb') as f:
  loaded_dict = pickle.load(f)
train_meta_info = loaded_dict['meta'] 
train_samples = loaded_dict['samples']

loaded_dict = {}
with open('test_samples.pkl', 'rb') as f:
  loaded_dict = pickle.load(f)
test_meta_info = loaded_dict['meta'] 
test_samples = loaded_dict['samples']
loaded_dict = {}

In [5]:
sums = np.sum(train_samples, axis=2)
train_samples = train_samples/ sums[:, :, np.newaxis]

sums = np.sum(test_samples, axis=2)
test_samples = test_samples/ sums[:, :, np.newaxis]

In [6]:
class LoadDataset(Dataset):
    def __init__(self, indices, train):
        self.indices = indices
        self.is_train = train

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, index):
        ind = self.indices[index]
        if self.is_train == True:
          dataY = torch.tensor(train_meta_info[ind][-1])
          dataX = torch.tensor(train_samples[ind]/ max_arr)
        else:
          dataY = torch.tensor(test_meta_info[ind][-1])
          dataX = torch.tensor(test_samples[ind]/ max_arr)
        dataX = dataX.float()
        dataY = dataY.float()
        return (dataX, dataY)

In [ ]:
class VGG_convnet(nn.Module):

    def __init__(self, bin_no = 282, feature_no = 156):

        super(VGG_convnet, self).__init__()

        self.NN = nn.Sequential(nn.Linear(feature_no, 64), nn.ReLU(),
                            nn.Linear(64, 96), nn.ReLU(),
                            nn.Linear(96, 128), nn.ReLU())
        
        self.pos_emb1D = torch.nn.Parameter(torch.randn(bin_no, 128))
        
        # block 1:         
        self.conv1a = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=3, padding=1)
        self.conv1b = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.LN1 = nn.LayerNorm(bin_no)
        
        # block 2:      
        self.conv2a = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv2b = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.LN2 = nn.LayerNorm(bin_no)
        
        # block 3:            
        self.conv3a =  nn.Conv1d(in_channels=64, out_channels=96, kernel_size=3, padding=1)
        self.conv3b =  nn.Conv1d(in_channels=96, out_channels=96, kernel_size=3, padding=1)
        self.LN3 = nn.LayerNorm(bin_no)
        
        #block 4:       
        self.conv4a = nn.Conv1d(in_channels=96, out_channels=128, kernel_size=3, padding=1)
        self.conv4b = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.LN4 = nn.LayerNorm(bin_no)
        
        # linear layers:
        self.Dense = nn.Sequential(nn.Linear(128, 128), nn.ReLU(),
                                   nn.Linear(128, 64), nn.ReLU(),
                                   nn.Linear(64, 1)) 

    def forward(self, x):
        x = self.NN(x)
        x = x + self.pos_emb1D
        x = x.permute(0, 2, 1)
        
        # block 1:  
        x = F.relu(self.conv1a(x))
        x = F.relu(self.conv1b(x))
        x = self.LN1(x)
        
        # block 2:   
        x = F.relu(self.conv2a(x))
        x = F.relu(self.conv2b(x))
        x = self.LN2(x)

        # block 3:    
        x = F.relu(self.conv3a(x))
        x = F.relu(self.conv3b(x))
        x = self.LN3(x)
        
        #block 4:      
        x = F.relu(self.conv4a(x))
        x = F.relu(self.conv4b(x))
        x = self.LN4(x)
        
        y = torch.mean(x, axis=2)
        
        # linear layers:   
        y = self.Dense(y)
        y = y.squeeze()
        
        return y

In [8]:
def custom_loss(scores, targets):
    const = 0.003
    numerator = const + torch.abs(scores - targets)
    denominator = const + targets
    total_loss = torch.sum(numerator/denominator)
    avg_loss = total_loss/ scores.size(0)
    return avg_loss

In [9]:
def metric_calc(loader, thr):
    running_MAE = 0
    num_batches = 0
    TP, TN, FP, FN = 0, 0, 0, 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(loader):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # forward
            scores = model(data)
            scores = scores.view(targets.size())
            
            # metric calc (MAE)
            #MAE_batch = L1criterion(scores, targets)
            MAE_batch = criterion(scores, targets)
            running_MAE = running_MAE + MAE_batch.item()
            
            # TP, TN, FP, FN
            for i in range(targets.size(0)):
                # if targets[i].item()>0.0 and targets[i].item()<0.01:
                if targets[i].item()>0.0 and targets[i].item()<thr:
                     continue
                elif scores[i].item()>thr and targets[i].item()>0.0:
                    TP += 1
                elif scores[i].item()<=thr and targets[i].item()>0.0:
                    FN+=1
                elif scores[i].item()>thr and targets[i].item()==0.0:
                    FP+=1
                elif scores[i].item()<=thr and targets[i].item()==0.0:
                    TN+=1

            num_batches+=1
    
    MAE_final = running_MAE/num_batches
    SN_final = TP/(TP+FN)
    SP_final = TN/(TN+FP)
    
    return MAE_final, SN_final, SP_final

In [10]:
def train(train_loader):
  for epoch in range(70):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        data.requires_grad_()
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        scores = scores.view(targets.size())
        # loss = criterion(scores, targets)
        loss = custom_loss(scores, targets)
        #loss = 2*loss1 + loss2

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

In [11]:
def make_csv(split_no, max_arr):
    csv_list = []
    test_y = []
    scores, sample_names = [], []
    
    model.eval()
    with torch.no_grad():
      for index in test_indices:
          sample_names.append(test_meta_info[index][1])
          dataX = np.copy(test_samples[index])
          dataX = dataX/ max_arr
          test_y.append(test_meta_info[index][-1])

          dataX = torch.tensor(dataX)
          # dataX = dataX.permute(1, 0)     ########
          dataX = torch.unsqueeze(dataX, dim=0)
          dataX = dataX.float()

          score = model(dataX.to(device))
          scores.append(score.item())
        
    for i in range(len(scores)):
        tmp = []
        tmp.append(sample_names[i])
        tmp.append(scores[i])
        tmp.append(test_y[i])
        csv_list.append(tmp)
        
    folder = '/content/drive/MyDrive/PhD/Fragle_TSS/Tested_Algorithms/Large Bin Based Modeling/10_split_csv/'
    filePath = folder + 'test' + str(split_no) + '.csv'
    my_df = pd.DataFrame(csv_list)
    my_df.to_csv(filePath, index=False, header=['Sample_ID', 'Pred_Fraction', 'True_Fraction'])

In [12]:
for N in range(10):
  split_train = dic['train'][N] + dic['val'][N]
  split_test = dic['test'][N]
  train_indices, test_indices = [], []
  max_arr = []

  for i in range(len(train_meta_info)):
    if train_meta_info[i][1] in split_train:
      train_indices.append(i)
  for i in range(len(test_meta_info)):   
    if test_meta_info[i][1] in split_test:
      test_indices.append(i)
  max_arr = np.max(train_samples[train_indices, :,:], axis=0)

  train_data = LoadDataset(train_indices, True)
  test_data = LoadDataset(test_indices, False)
  train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
  test_loader = DataLoader(dataset=test_data, batch_size=32)

  # initializing model for split N
  model = VGG_convnet().to(device)
  criterion = nn.L1Loss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)

  # train the model
  print(f'Split no. {N}')
  train(train_loader) ###

  # model_path = '../../Experiments/Models/model' + str(N) + '.pt'
  # torch.save(model.state_dict(), model_path)

  make_csv(N+1, max_arr)

  for thr in thresholds:
      MAE, SN, SP = metric_calc(test_loader, thr)
      print(f'threshold: {thr*100}%, MAE: {MAE}, Sensitivity: {SN}, Specificity: {SP}')
      MAE_dic[thr].append(MAE)
      SN_dic[thr].append(SN)
      SP_dic[thr].append(SP)
  print()
  print()

Split no. 0
threshold: 5.0%, MAE: 0.04374711621891369, Sensitivity: 0.8557046979865772, Specificity: 1.0
threshold: 3.0%, MAE: 0.04374711621891369, Sensitivity: 0.868421052631579, Specificity: 1.0
threshold: 2.0%, MAE: 0.04374711621891369, Sensitivity: 0.868663594470046, Specificity: 0.9629629629629629
threshold: 1.0%, MAE: 0.04374711621891369, Sensitivity: 0.8831967213114754, Specificity: 0.9629629629629629
threshold: 0.5%, MAE: 0.04374711621891369, Sensitivity: 0.8737864077669902, Specificity: 0.9259259259259259
threshold: 0.1%, MAE: 0.04374711621891369, Sensitivity: 0.8312829525483304, Specificity: 0.8518518518518519


Split no. 1
threshold: 5.0%, MAE: 0.041983038102361286, Sensitivity: 0.7781456953642384, Specificity: 1.0
threshold: 3.0%, MAE: 0.041983038102361286, Sensitivity: 0.7596899224806202, Specificity: 1.0
threshold: 2.0%, MAE: 0.041983038102361286, Sensitivity: 0.7945823927765236, Specificity: 0.9629629629629629
threshold: 1.0%, MAE: 0.041983038102361286, Sensitivity: 0.82

In [13]:
for thr in thresholds:
    print(f'At threshold of {thr*100}%')
    mean_MAE = round(np.mean(MAE_dic[thr]), 3)
    med_MAE = round(np.median(MAE_dic[thr]), 3)
    std_MAE = round(np.std(MAE_dic[thr]), 3)
    print(f'MAE mean: {mean_MAE}, median: {med_MAE}, std: {std_MAE}')
        
    mean_SN = round(np.mean(SN_dic[thr]), 3)
    med_SN = round(np.median(SN_dic[thr]), 3)
    std_SN = round(np.std(SN_dic[thr]), 3)
    print(f'Sensitivity mean: {mean_SN}, median: {med_SN}, std: {std_SN}')
    
    mean_SP = round(np.mean(SP_dic[thr]), 3)
    med_SP = round(np.median(SP_dic[thr]), 3)
    std_SP = round(np.std(SP_dic[thr]), 3)
    print(f'Specificity mean: {mean_SP}, median: {med_SP}, std: {std_SP}')
    
    print()

At threshold of 5.0%
MAE mean: 0.04, median: 0.04, std: 0.003
Sensitivity mean: 0.834, median: 0.836, std: 0.041
Specificity mean: 1.0, median: 1.0, std: 0.0

At threshold of 3.0%
MAE mean: 0.04, median: 0.04, std: 0.003
Sensitivity mean: 0.847, median: 0.869, std: 0.04
Specificity mean: 0.989, median: 1.0, std: 0.017

At threshold of 2.0%
MAE mean: 0.04, median: 0.04, std: 0.003
Sensitivity mean: 0.87, median: 0.875, std: 0.031
Specificity mean: 0.967, median: 0.963, std: 0.011

At threshold of 1.0%
MAE mean: 0.04, median: 0.04, std: 0.003
Sensitivity mean: 0.897, median: 0.901, std: 0.029
Specificity mean: 0.889, median: 0.889, std: 0.083

At threshold of 0.5%
MAE mean: 0.04, median: 0.04, std: 0.003
Sensitivity mean: 0.909, median: 0.919, std: 0.029
Specificity mean: 0.815, median: 0.852, std: 0.131

At threshold of 0.1%
MAE mean: 0.04, median: 0.04, std: 0.003
Sensitivity mean: 0.9, median: 0.902, std: 0.038
Specificity mean: 0.637, median: 0.63, std: 0.155

